<a href="https://colab.research.google.com/github/parus-cristatus/tolokapizza/blob/main/tolokaapi/get_responses_without_banned_performers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Docs:**

Get responses: https://toloka.ai/docs/api/result/

Get bans: https://toloka.ai/docs/api/ban-get-list/

* Bans are considered at the moment of getting assignments and are checked once. If a ban has expiration date and it has already expired - there will be an empty array in the response. Project ban takes into consideration current ban on **any** pool in the project.

In [56]:
import requests
from csv import DictWriter

In [ ]:
endpoint = "https://sandbox.toloka.dev/api/v1/" # https://toloka.dev/api/v1/ for Production
token = input("Please enter your token: ") # Toloka OAuth token

In [57]:
def get_call(call: str, token: str, endpoint: str, params: dict=None):
    headers = {
        "Authorization": "OAuth " + token
    }
    try:
        response = requests.get(f"{endpoint}{call}", headers=headers, params=params)
        return response.json()
    except requests.exceptions.RequestException as error:
        print(error)

In [58]:
pool = "1476980"
project = "136531"

In [59]:
def get_assignments_params(pool, limit, status: str="ACCEPTED", sort: str="id", id_gt: str=None) ->dict:
    return {
        "pool_id": pool,
        "limit": limit,
        "status": status,
        "sort": sort,
        "id_gt": id_gt
    } 

In [60]:
# can change to PROJECT scope with project ID
def get_bans_params(pool, user_id, scope: str="POOL") ->dict:
    return {
        "pool_id": pool,
        "user_id": user_id,
        "scope": scope
    } 

In [73]:
banned_users = set()
not_banned_users = set()

def sort_user(user):
    if user not in banned_users and user not in not_banned_users:
        bans = get_call("user-restrictions", token, endpoint, params=get_bans_params(pool, user))
        if not bans["items"]:
            not_banned_users.add(user)
        else:
            banned_users.add(user)

In [74]:
field_names = ["assignment", "user", "task", "input", "output", "golden"]

def write_to_csv(data: dict):
        with open("output.csv", "a") as output:
            dictwriter_object = DictWriter(output, fieldnames=field_names)
            dictwriter_object.writerow(data)

In [75]:
def create_csv_row(assignment):
    for idx, task in enumerate(assignment["tasks"]):
        data = {
                "assignment": assignment["id"],
                "user": assignment["user_id"],
                "task": assignment["tasks"][idx]["id"],
                "input": assignment["tasks"][idx]["input_values"],
                "output": assignment["solutions"][idx]["output_values"],
                "golden": "golden" if "known_solutions" in task else None
                }

        write_to_csv(data)

In [76]:
def get_assignments(limit: int=300):
    assignments = get_call("assignments", token, endpoint, params=get_assignments_params(pool, limit))
    last_id = assignments["items"][-1]["id"]
    has_more = assignments["has_more"]

    while (True):
        if has_more:
            for assignment in assignments["items"]:

                user = assignment["user_id"]
                sort_user(user)

                if user in not_banned_users:
                    create_csv_row(assignment)

            # repeat API call if there're more assignments
            assignments = get_call('assignments', token, endpoint, params=get_assignments_params(pool, limit, id_gt=last_id))
            last_id = assignments['items'][-1]['id']
            has_more = assignments['has_more']
        else:

            # If has_more is False then process returned assignments and that's all
            for assignment in assignments["items"]:

                user = assignment["user_id"]
                sort_user(user)

                if user in not_banned_users:
                    create_csv_row(assignment)

            break

In [ ]:
get_assignments()
print(f"Banned users: {banned_users}, Not banned users: {not_banned_users}")